# Dynamic Programming: LCS, LIC and Coins

## Revision

![Revision notes](./02-1-revision.png)

# LCS: From Recursion to Dynamic Programming

## 🎯 Problem Statement

**Longest Common Subsequence (LCS)**: Given two strings A and B, find the longest sequence of characters that appears in both strings in the same order (but not necessarily consecutive).

**Example**:
- A = `"stone"`
- B = `"longest"`
- LCS = `"one"` (length 3)

---

## 🌳 Approach 1: Naive Recursion

### 🧠 **The Recursive Thinking**

At each position (i,j), we ask: **"Do the current characters match?"**

**If they match**: Include this character + solve for the rest
**If they don't match**: Try skipping either character and take the better result

### 📝 **Recursive Definition**

```
LCS(i,j) = LCS of suffixes A[i:] and B[j:]

Base case: 
- If i >= len(A) or j >= len(B): return 0

Recursive case:
- If A[i] == B[j]: return 1 + LCS(i+1, j+1)  
- Else: return max(LCS(i+1, j), LCS(i, j+1))
```

### 💻 **Code: Naive Recursion**

```python
def lcs_recursive(A, B, i=0, j=0):
    # Base case: reached end of either string
    if i >= len(A) or j >= len(B):
        return 0
    
    # If characters match
    if A[i] == B[j]:
        return 1 + lcs_recursive(A, B, i+1, j+1)
    else:
        # Try both options: skip A[i] or skip B[j]
        return max(lcs_recursive(A, B, i+1, j),    # Skip A[i]
                  lcs_recursive(A, B, i, j+1))     # Skip B[j]

# Test
A = "bda"
B = "abcd"
print(f"LCS length: {lcs_recursive(A, B)}")
```

### 🌳 **Recursion Tree Visualization**

![INSERT IMAGE 1 HERE](./02-2-recursion.png)

### 😱 **The Problem: Exponential Time!**

**Time Complexity**: O(2^(m+n)) where m=len(A), n=len(B)

**Why so slow?** The same subproblems are solved multiple times!

Example: `LCS(1,1)` might be computed many times in different branches.

---

## 🧠 Approach 2: Recursion with Memoization

### 💡 **The Fix: Remember What We've Computed**

Same recursive logic, but store results in a memo table to avoid recomputation.

### 💻 **Code: Memoized Recursion**

```python
def lcs_memo(A, B):
    memo = {}  # Dictionary to store computed results
    
    def helper(i, j):
        # Check if already computed
        if (i, j) in memo:
            return memo[(i, j)]
        
        # Base case
        if i >= len(A) or j >= len(B):
            result = 0
        # If characters match
        elif A[i] == B[j]:
            result = 1 + helper(i+1, j+1)
        else:
            # Try both options
            result = max(helper(i+1, j), helper(i, j+1))
        
        # Store result before returning
        memo[(i, j)] = result
        return result
    
    return helper(0, 0)

# Test
A = "bda"
B = "abcd"
print(f"LCS length: {lcs_memo(A, B)}")
print(f"Memo table: {memo}")  # See what was computed
```

![INSERT IMAGE 2 HERE](./02-3-recursion-with-memo-table.png)
![INSERT IMAGE 3 HERE](./02-4-recursion-fast.png)


### ⚡ **Time Complexity: O(m×n)**

Now each subproblem is solved exactly once!

---

## 📊 Approach 3: Bottom-Up Dynamic Programming

### 🎯 **The Ultimate Optimization: Build Table Systematically**

Instead of recursing from top, build the solution from bottom-up using a 2D table.

### 🏗️ **DP Table Setup**

**Table Definition**: `dp[i][j]` = LCS length of `A[i:]` and `B[j:]`

**Dimensions**: `(len(A)+1) × (len(B)+1)`

### 📋 **Step-by-Step DP Algorithm**

```python
def lcs_dp(A, B):
    m, n = len(A), len(B)
    
    # Create DP table with extra row/column for empty strings
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Fill table backwards (from bottom-right to top-left)
    for i in range(m - 1, -1, -1):      # i goes: m-1, m-2, ..., 0
        for j in range(n - 1, -1, -1):  # j goes: n-1, n-2, ..., 0
            if A[i] == B[j]:
                dp[i][j] = 1 + dp[i+1][j+1]
            else:
                dp[i][j] = max(dp[i+1][j], dp[i][j+1])
    
    return dp[0][0], dp  # Return both result and table

# Test with simple example
A = "bd"
B = "abcd"
result, table = lcs_dp(A, B)
print(f"LCS length: {result}")
```

---

## 🔍 Example 1: Simple Case

**Strings**: A = `"bd"`, B = `"abcd"`

### 📊 **Initial DP Table**
```
      j:  0   1   2   3   4
         'a' 'b' 'c' 'd' ''
    i:  
    0 'b'  ?   ?   ?   ?   0
    1 'd'  ?   ?   ?   ?   0
    2 ''   0   0   0   0   0
```

### 🔄 **Filling Process**

**Step 1**: `dp[1][3]` - Compare `A[1]='d'` vs `B[3]='d'`
- **Match!** → `dp[1][3] = 1 + dp[2][4] = 1 + 0 = 1`

**Step 2**: `dp[1][2]` - Compare `A[1]='d'` vs `B[2]='c'`  
- **No match** → `dp[1][2] = max(dp[2][2], dp[1][3]) = max(0, 1) = 1`

**Step 3**: `dp[1][1]` - Compare `A[1]='d'` vs `B[1]='b'`
- **No match** → `dp[1][1] = max(dp[2][1], dp[1][2]) = max(0, 1) = 1`

**Step 4**: `dp[1][0]` - Compare `A[1]='d'` vs `B[0]='a'`
- **No match** → `dp[1][0] = max(dp[2][0], dp[1][1]) = max(0, 1) = 1`

**Step 5**: `dp[0][3]` - Compare `A[0]='b'` vs `B[3]='d'`
- **No match** → `dp[0][3] = max(dp[1][3], dp[0][4]) = max(1, 0) = 1`

**Step 6**: `dp[0][2]` - Compare `A[0]='b'` vs `B[2]='c'`
- **No match** → `dp[0][2] = max(dp[1][2], dp[0][3]) = max(1, 1) = 1`

**Step 7**: `dp[0][1]` - Compare `A[0]='b'` vs `B[1]='b'`
- **Match!** → `dp[0][1] = 1 + dp[1][2] = 1 + 1 = 2`

**Step 8**: `dp[0][0]` - Compare `A[0]='b'` vs `B[0]='a'`
- **No match** → `dp[0][0] = max(dp[1][0], dp[0][1]) = max(1, 2) = 2`

### 📊 **Final DP Table**
```
      j:  0   1   2   3   4
         'a' 'b' 'c' 'd' ''
    i:  
    0 'b'  2   2   1   1   0
    1 'd'  1   1   1   1   0
    2 ''   0   0   0   0   0
```

![IMAGE](./02-5-LCS-2.png)

**Result**: LCS length = `dp[0][0] = 2`
**LCS**: `"bd"` (both characters from A appear in B)

---

## 🌟 Example 2: More Complex Case

**Strings**: A = `"stone"`, B = `"longest"`

![IMAGE](./02-5-LCS-3.png)
![IMAGE](./02-5-LCS-4.png)


### 💻 **Complete Code with Visualization**

```python
def lcs_dp_detailed(A, B):
    m, n = len(A), len(B)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    print(f"A = '{A}' (length {m})")
    print(f"B = '{B}' (length {n})")
    print(f"DP table size: {m+1} x {n+1}")
    print()
    
    # Fill the table
    for i in range(m - 1, -1, -1):
        for j in range(n - 1, -1, -1):
            if A[i] == B[j]:
                dp[i][j] = 1 + dp[i+1][j+1]
                print(f"Match: A[{i}]='{A[i]}' == B[{j}]='{B[j]}' → dp[{i}][{j}] = {dp[i][j]}")
            else:
                dp[i][j] = max(dp[i+1][j], dp[i][j+1])
                print(f"No match: A[{i}]='{A[i]}' != B[{j}]='{B[j]}' → dp[{i}][{j}] = max({dp[i+1][j]}, {dp[i][j+1]}) = {dp[i][j]}")
    
    return dp[0][0], dp

def print_dp_table(A, B, dp):
    m, n = len(A), len(B)
    
    # Print header
    print("      j:", end="")
    for j in range(n + 1):
        print(f"  {j:2}", end="")
    print()
    
    print("        ", end="")
    for j in range(n):
        print(f" '{B[j]}'", end="")
    print(" ''")
    
    # Print rows
    for i in range(m + 1):
        if i < m:
            print(f"  {i:2} '{A[i]}'", end="")
        else:
            print(f"  {i:2} '' ", end="")
        
        for j in range(n + 1):
            print(f"  {dp[i][j]:2}", end="")
        print()

# Run the example
A = "stone"
B = "longest"
length, table = lcs_dp_detailed(A, B)
print("\nFinal DP Table:")
print_dp_table(A, B, table)
print(f"\nLCS Length: {length}")
```

![IMAGE]()

### 🎯 **LCS Reconstruction**

```python
def reconstruct_lcs(A, B, dp):
    lcs = []
    i, j = 0, 0
    
    while i < len(A) and j < len(B):
        if A[i] == B[j]:
            lcs.append(A[i])
            i += 1
            j += 1
        elif dp[i+1][j] > dp[i][j+1]:
            i += 1
        else:
            j += 1
    
    return ''.join(lcs)

# Get the actual LCS string
A = "stone"
B = "longest"
_, table = lcs_dp(A, B)
lcs_string = reconstruct_lcs(A, B, table)
print(f"LCS: '{lcs_string}'")
```

![IMAGE](./02-5-LCS-1.png)

---

## 📈 Complexity Comparison

| **Approach** | **Time Complexity** | **Space Complexity** | **Pros** | **Cons** |
|-------------|-------------------|-------------------|----------|----------|
| **Naive Recursion** | O(2^(m+n)) | O(m+n) | Simple to understand | Exponentially slow |
| **Memoized Recursion** | O(m×n) | O(m×n) | Easy to implement | Uses recursion stack |
| **Bottom-Up DP** | O(m×n) | O(m×n) | Most efficient | Requires understanding DP |

---

## 🎯 Key Insights

### 🧠 **Why DP Works**
1. **Optimal Substructure**: LCS of A and B uses optimal LCS of suffixes
2. **Overlapping Subproblems**: Same (i,j) pairs computed multiple times in recursion
3. **Memoization**: Eliminates redundant computation
4. **Bottom-Up**: Builds solution systematically without recursion overhead

### 🔑 **Pattern Recognition**
- **2D DP**: When dealing with two sequences/strings
- **Backwards filling**: When subproblems depend on "future" positions
- **Choice at each step**: Match vs skip character(s)

### 🌟 **When to Use LCS**
- DNA sequence alignment
- File difference tools (diff)
- Version control systems
- Text similarity measurement
- Longest palindromic subsequence (string vs its reverse)

---

## 🚀 Extensions and Variations

1. **Print all LCS**: Modify reconstruction to find all possible LCS
2. **Space optimization**: Use only two rows instead of full table → O(min(m,n)) space
3. **Longest Common Substring**: Require consecutive characters
4. **Edit Distance**: Count operations needed to transform one string to another
5. **Multiple sequences**: Extend to 3 or more strings

The beauty of DP is how it transforms an exponential problem into a polynomial one by being smart about what we compute and remember!